In [36]:
import matplotlib.pyplot as plt
%matplotlib inline

import io_tools as iot
import os
import numpy as np
from time import time
import plyvel
from caffe import io
import sys

import cPickle as pickle
import gzip

from transformers import *

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import StratifiedKFold

# DATA_DIR = iot.DATA_DIR
DATA_DIR = '/media/raid_arr/data/otto/data/'
DB_OUT_TRAIN = os.path.join(DATA_DIR, 'train_lvl')
DB_OUT_TEST = os.path.join(DATA_DIR, 'test_lvl')
DB_OUT_ALL = os.path.join(DATA_DIR, 'all_lvl')
DB_OUT_ALL0 = os.path.join(DATA_DIR, 'all0_lvl')

In [21]:
try:
    ids_train, feats_train, labels_train
except NameError:
    ids_train = np.load(os.path.join(iot.DATA_DIR, 'train_ids.npy'))
    feats_train = np.load(os.path.join(iot.DATA_DIR, 'train_feats.npy')).astype(float)
    labels_train = np.load(os.path.join(iot.DATA_DIR, 'train_labels_enc.npy'))

try:
    ids_test, feats_test
except NameError:
    ids_test = np.load(os.path.join(iot.DATA_DIR, 'test_ids.npy'))
    feats_test = np.load(os.path.join(iot.DATA_DIR, 'test_feats.npy')).astype(float)   
    
skf = StratifiedKFold(labels_train, n_folds=5, shuffle=True)
# All
feats_all = np.r_[feats_train, feats_test]
labels_all = np.r_[labels_train, -1*np.ones(len(ids_test))].astype(int)

# All minus test0
train_ind, val_ind = iter(skf).next()
feats_all0 = np.r_[feats_train[train_ind, :], feats_test]
labels_all0 = np.r_[labels_train[train_ind], -1*np.ones(len(ids_test))].astype(int)


feats_fold_train = feats_train[train_ind, :]
labels_fold_train = labels_train[train_ind]
feats_fold_val = feats_train[val_ind, :]
labels_fold_val = labels_train[val_ind]


In [10]:
pipe = make_pipeline(LogTformer(), StandardScaler())
pipe.fit(np.r_[feats_train[train_ind, :], feats_test])

Pipeline(steps=[('logtformer', <__main__.LogTformer object at 0x7f980c89d650>), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True))])

In [30]:
pipe = make_pipeline(
    make_union(
        IdentityTformer(), LogTformer(), AnscombeTformer(), InverseTformer(),
    ), StandardScaler()
                    )
pipe.fit(np.r_[feats_train[train_ind, :], feats_test])


Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('identitytformer', <__main__.IdentityTformer object at 0x7f97c2bdb250>), ('logtformer', <__main__.LogTformer object at 0x7f97c2bdbcd0>), ('anscombetformer', <__main__.AnscombeTformer object at 0x7f97c2bdb2d0>), ('inversetformer', <__main__.InverseTformer object at 0x7f97c2bdb690>)],
       transformer_weights=None)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True))])

In [153]:
pipe = make_pipeline(
                    TfidfTransformer(norm=u'l2', 
                                      use_idf=True, 
                                      smooth_idf=True, 
                                      sublinear_tf=True),
                     DenseTformer(),
                     make_union(
                                IdentityTformer(),
#                                 FactorAnalysis(n_components=74),
#                                 PCA(n_components=20, whiten=True),
                                NzTformer(),
                                NzvarTformer(),
                                NzmeanTformer(),
                     ),
                     StandardScaler(),
#                      MinMaxScaler(),
                     )
pipe.fit(np.r_[feats_train[train_ind, :], feats_test])

/usr/lib/python2.7/site-packages/numpy/core/_methods.py:83: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/usr/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


Pipeline(steps=[('tfidftransformer', TfidfTransformer(norm=u'l2', smooth_idf=True, sublinear_tf=True, use_idf=True)), ('densetformer', <__main__.DenseTformer object at 0x7f1da68c4090>), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('identitytformer', <__main__.IdentityTformer object at 0x...former_weights=None)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True))])

In [31]:
X_train = pipe.transform(feats_fold_train)
y_train = labels_fold_train

X_valid = pipe.transform(feats_fold_val)
y_valid = labels_fold_val

X_test = pipe.transform(feats_test)
y_test = -1*np.ones(len(feats_test), dtype=int)

data = ((X_train, y_train),
        (X_valid, y_valid),
        (X_test, y_test))

In [32]:
X_test.shape

(144368, 372)

In [33]:
pickle.dump(data, gzip.open('/media/raid_arr/data/otto/data/iden-log-anscombe-inv_data.pgz', 'wb'))

In [8]:
def my_load():
    
#     X_train = pipe.transform(feats_fold_train)
    y_train = labels_fold_train

#     X_valid = pipe.transform(feats_fold_val)
    y_valid = labels_fold_val

#     X_test = pipe.transform(feats_test)
    y_test = -1*np.ones(len(feats_test), dtype=int)

    data = ((X_train, y_train),
            (X_valid, y_valid),
            (X_test, y_test))
    return data
    

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

# LESGO

In [155]:
from __future__ import print_function

import gzip
import itertools
import pickle
import os
import sys
import numpy as np
import lasagne
import theano
import theano.tensor as T
import time

PY2 = sys.version_info[0] == 2

if PY2:
    from urllib import urlretrieve

    def pickle_load(f, encoding):
        return pickle.load(f)
else:
    from urllib.request import urlretrieve

    def pickle_load(f, encoding):
        return pickle.load(f, encoding=encoding)

DATA_URL = 'http://deeplearning.net/data/mnist/mnist.pkl.gz'
DATA_FILENAME = 'mnist.pkl.gz'

NUM_EPOCHS = 5000
BATCH_SIZE = 2048
NUM_HIDDEN_UNITS = 1024
LEARNING_RATE = 0.01
MOMENTUM = 0.9

In [151]:
def _load_data(url=DATA_URL, filename=DATA_FILENAME):
    if not os.path.exists(filename):
        print("Downloading MNIST")
        urlretrieve(url, filename)

    with gzip.open(filename, 'rb') as f:
        return pickle_load(f, encoding='latin-1')


def load_data():
#     data = _load_data()
    data = my_load()
    X_train, y_train = data[0]
    X_valid, y_valid = data[1]
    X_test, y_test = data[2]

    return dict(
        X_train=theano.shared(lasagne.utils.floatX(X_train)),
        y_train=T.cast(theano.shared(y_train), 'int32'),
        X_valid=theano.shared(lasagne.utils.floatX(X_valid)),
        y_valid=T.cast(theano.shared(y_valid), 'int32'),
        X_test=theano.shared(lasagne.utils.floatX(X_test)),
        y_test=T.cast(theano.shared(y_test), 'int32'),
        num_examples_train=X_train.shape[0],
        num_examples_valid=X_valid.shape[0],
        num_examples_test=X_test.shape[0],
        input_dim=X_train.shape[1],
        output_dim=len(np.unique(y_train)),
    )


def build_model(input_dim, output_dim,
                batch_size=BATCH_SIZE, num_hidden_units=NUM_HIDDEN_UNITS):

    l_in = lasagne.layers.InputLayer(
        shape=(batch_size, input_dim),
    )
    l_hidden1 = lasagne.layers.DenseLayer(
        l_in,
        num_units=num_hidden_units,
        nonlinearity=lasagne.nonlinearities.rectify,
#         nonlinearity=lasagne.nonlinearities.identity,
    )
    
#     lh1m = lasagne.layers.FeaturePoolLayer(
#         l_hidden1,
#         ds=2,
#     )
    
    l_hidden1_dropout = lasagne.layers.DropoutLayer(
        l_hidden1,
        p=0.5,
    )

    l_hidden2 = lasagne.layers.DenseLayer(
        l_hidden1_dropout,
        num_units=num_hidden_units,
        nonlinearity=lasagne.nonlinearities.rectify,
#         nonlinearity=lasagne.nonlinearities.identity,
    )
    
#     lh2m = lasagne.layers.FeaturePoolLayer(
#         l_hidden2,
#         ds=2,
#     )
    
    
    l_hidden2_dropout = lasagne.layers.DropoutLayer(
        l_hidden2,
        p=0.5,
    )
    l_out = lasagne.layers.DenseLayer(
        l_hidden2_dropout,
        num_units=output_dim,
        nonlinearity=lasagne.nonlinearities.softmax,
    )
    return l_out


def create_iter_functions(dataset, output_layer,
                          X_tensor_type=T.matrix,
                          batch_size=BATCH_SIZE,
                          learning_rate=LEARNING_RATE, momentum=MOMENTUM):
    batch_index = T.iscalar('batch_index')
    X_batch = X_tensor_type('x')
    y_batch = T.ivector('y')
    batch_slice = slice(
        batch_index * batch_size, (batch_index + 1) * batch_size)

    objective = lasagne.objectives.Objective(output_layer,
        loss_function=lasagne.objectives.categorical_crossentropy)

    loss_train = objective.get_loss(X_batch, target=y_batch)
    loss_eval = objective.get_loss(X_batch, target=y_batch,
                                   deterministic=True)

    pred = T.argmax(
        output_layer.get_output(X_batch, deterministic=True), axis=1)
    accuracy = T.mean(T.eq(pred, y_batch), dtype=theano.config.floatX)

    all_params = lasagne.layers.get_all_params(output_layer)
    updates = lasagne.updates.nesterov_momentum(
        loss_train, all_params, learning_rate, momentum)

    iter_train = theano.function(
        [batch_index], loss_train,
        updates=updates,
        givens={
            X_batch: dataset['X_train'][batch_slice],
            y_batch: dataset['y_train'][batch_slice],
        },
    )

    iter_valid = theano.function(
        [batch_index], [loss_eval, accuracy],
        givens={
            X_batch: dataset['X_valid'][batch_slice],
            y_batch: dataset['y_valid'][batch_slice],
        },
    )

    iter_test = theano.function(
        [batch_index], [loss_eval, accuracy],
        givens={
            X_batch: dataset['X_test'][batch_slice],
            y_batch: dataset['y_test'][batch_slice],
        },
    )

    return dict(
        train=iter_train,
        valid=iter_valid,
        test=iter_test,
    )


def train(iter_funcs, dataset, batch_size=BATCH_SIZE):
    num_batches_train = dataset['num_examples_train'] // batch_size
    num_batches_valid = dataset['num_examples_valid'] // batch_size

    for epoch in itertools.count(1):
        batch_train_losses = []
        for b in range(num_batches_train):
            batch_train_loss = iter_funcs['train'](b)
            batch_train_losses.append(batch_train_loss)

        avg_train_loss = np.mean(batch_train_losses)

        batch_valid_losses = []
        batch_valid_accuracies = []
        for b in range(num_batches_valid):
            batch_valid_loss, batch_valid_accuracy = iter_funcs['valid'](b)
            batch_valid_losses.append(batch_valid_loss)
            batch_valid_accuracies.append(batch_valid_accuracy)

        avg_valid_loss = np.mean(batch_valid_losses)
        avg_valid_accuracy = np.mean(batch_valid_accuracies)

        yield {
            'number': epoch,
            'train_loss': avg_train_loss,
            'valid_loss': avg_valid_loss,
            'valid_accuracy': avg_valid_accuracy,
        }


def main(num_epochs=NUM_EPOCHS):
    print("Loading data...")
    dataset = load_data()

    print("Building model and compiling functions...")
    output_layer = build_model(
        input_dim=dataset['input_dim'],
        output_dim=dataset['output_dim'],
    )
    iter_funcs = create_iter_functions(dataset, output_layer)

    print("Starting training...")
    now = time.time()
    try:
        for epoch in train(iter_funcs, dataset):
            if (epoch['number']-1) % 100 == 0:
#             if 1:
                print("Epoch {} of {} took {:.3f}s".format(
                    epoch['number'], num_epochs, time.time() - now))
                now = time.time()
                print("  training loss:\t\t{:.6f}".format(epoch['train_loss']))
                print("  validation loss:\t\t{:.6f}".format(epoch['valid_loss']))
                print("  validation accuracy:\t\t{:.2f} %%".format(
                    epoch['valid_accuracy'] * 100))
                sys.stdout.flush()
            if epoch['number'] >= num_epochs:
                break

    except KeyboardInterrupt:
        pass

    return output_layer

In [156]:
out_layer = main()

Loading data...
Building model and compiling functions...
Starting training...
Epoch 1 of 5000 took 0.444s
  training loss:		2.877856
  validation loss:		1.982359
  validation accuracy:		28.91 %%
Epoch 101 of 5000 took 44.289s
  training loss:		0.613216
  validation loss:		0.557774
  validation accuracy:		79.12 %%
Epoch 201 of 5000 took 44.288s
  training loss:		0.534340
  validation loss:		0.517341
  validation accuracy:		80.06 %%
Epoch 301 of 5000 took 44.296s
  training loss:		0.478634
  validation loss:		0.497502
  validation accuracy:		80.54 %%
Epoch 401 of 5000 took 44.307s
  training loss:		0.440227
  validation loss:		0.487613
  validation accuracy:		80.79 %%
Epoch 501 of 5000 took 44.310s
  training loss:		0.406042
  validation loss:		0.482963
  validation accuracy:		81.09 %%
Epoch 601 of 5000 took 44.306s
  training loss:		0.376894
  validation loss:		0.483602
  validation accuracy:		81.14 %%
Epoch 701 of 5000 took 44.305s
  training loss:		0.350270
  validation loss:		0.4869

In [31]:
# data = _load_data()
data = my_load()
X_train, y_train = data[0]
X_valid, y_valid = data[1]
X_test, y_test = data[2]

In [148]:
feats_train.shape

(61878, 93)